In [1]:
import pandas as pd

In [2]:
# Determine file encoding format to ensure we can read this date into a dataframe
with open("data/ProdVolumeQuarters.csv") as file:
    print(file)

<_io.TextIOWrapper name='data/ProdVolumeQuarters.csv' mode='r' encoding='cp1252'>


In [3]:
# observing this datafile we view observations and features
df = pd.read_csv ("data/ProdVolumeQuarters.csv")
df.shape

(4, 59)

In [4]:
# list all the features of the datset
print (df.columns.values)

['freq' 'indic_bt' 'nace_r2' 's_adj' 'unit' 'geo\\TIME_PERIOD' '2010-Q1 '
 '2010-Q2 ' '2010-Q3 ' '2010-Q4 ' '2011-Q1 ' '2011-Q2 ' '2011-Q3 '
 '2011-Q4 ' '2012-Q1 ' '2012-Q2 ' '2012-Q3 ' '2012-Q4 ' '2013-Q1 '
 '2013-Q2 ' '2013-Q3 ' '2013-Q4 ' '2014-Q1 ' '2014-Q2 ' '2014-Q3 '
 '2014-Q4 ' '2015-Q1 ' '2015-Q2 ' '2015-Q3 ' '2015-Q4 ' '2016-Q1 '
 '2016-Q2 ' '2016-Q3 ' '2016-Q4 ' '2017-Q1 ' '2017-Q2 ' '2017-Q3 '
 '2017-Q4 ' '2018-Q1 ' '2018-Q2 ' '2018-Q3 ' '2018-Q4 ' '2019-Q1 '
 '2019-Q2 ' '2019-Q3 ' '2019-Q4 ' '2020-Q1 ' '2020-Q2 ' '2020-Q3 '
 '2020-Q4 ' '2021-Q1 ' '2021-Q2 ' '2021-Q3 ' '2021-Q4 ' '2022-Q1 '
 '2022-Q2 ' '2022-Q3 ' '2022-Q4 ' '2023-Q1 ']


In [5]:
# Drop features we are not interested in for this exercise
df = df.drop(['freq', 'indic_bt', 'nace_r2', 's_adj', 'unit'], axis = 1)
df.head()

,geo\TIME_PERIOD,2010-Q1,2010-Q2,2010-Q3,2010-Q4,2011-Q1,2011-Q2,2011-Q3,2011-Q4,2012-Q1,...,2020-Q4,2021-Q1,2021-Q2,2021-Q3,2021-Q4,2022-Q1,2022-Q2,2022-Q3,2022-Q4,2023-Q1
0,AT,97.8,98.9,97.1,96.0,96.8,99.9,98.5,101.3,98.7,...,123.2,126.7,127.7,125.9,126.6,131.8,132.6,129.4,130.6,:
1,BE,101.0,102.7,102.9,97.5,106.5,106.0,106.2,106.7,105.6,...,96.2,99.8,98.2,97.8,96.9,99.3,99.0,99.3,99.3,:
2,IE,99.8,98.9,93.0,85.0,79.0,76.3,78.3,81.1,77.4,...,140.3,110.0,123.1,131.8,140.6,135.5,131.5,129.1,125.1 p,:
3,NO,78.5,79.8,80.1,79.3,79.6,81.3,80.2,85.2,86.3,...,116.7,117.8,116.8,118.2,121.3,123.2,124.1,123.0,123.9,123.7


In [6]:
df = df.rename(columns=lambda x: x.strip())

In [7]:
# list all the features of the datset
print(list(df.columns))

['geo\\TIME_PERIOD', '2010-Q1', '2010-Q2', '2010-Q3', '2010-Q4', '2011-Q1', '2011-Q2', '2011-Q3', '2011-Q4', '2012-Q1', '2012-Q2', '2012-Q3', '2012-Q4', '2013-Q1', '2013-Q2', '2013-Q3', '2013-Q4', '2014-Q1', '2014-Q2', '2014-Q3', '2014-Q4', '2015-Q1', '2015-Q2', '2015-Q3', '2015-Q4', '2016-Q1', '2016-Q2', '2016-Q3', '2016-Q4', '2017-Q1', '2017-Q2', '2017-Q3', '2017-Q4', '2018-Q1', '2018-Q2', '2018-Q3', '2018-Q4', '2019-Q1', '2019-Q2', '2019-Q3', '2019-Q4', '2020-Q1', '2020-Q2', '2020-Q3', '2020-Q4', '2021-Q1', '2021-Q2', '2021-Q3', '2021-Q4', '2022-Q1', '2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1']


In [10]:
df = df.rename (columns={'geo\TIME_PERIOD': 'GEO', '2010-Q1 ': '2010Q1', '2010-Q2': '2010Q2'})
print(list(df.columns))
#df.head()

['GEO', '2010-Q1', '2010Q2', '2010-Q3', '2010-Q4', '2011-Q1', '2011-Q2', '2011-Q3', '2011-Q4', '2012-Q1', '2012-Q2', '2012-Q3', '2012-Q4', '2013-Q1', '2013-Q2', '2013-Q3', '2013-Q4', '2014-Q1', '2014-Q2', '2014-Q3', '2014-Q4', '2015-Q1', '2015-Q2', '2015-Q3', '2015-Q4', '2016-Q1', '2016-Q2', '2016-Q3', '2016-Q4', '2017-Q1', '2017-Q2', '2017-Q3', '2017-Q4', '2018-Q1', '2018-Q2', '2018-Q3', '2018-Q4', '2019-Q1', '2019-Q2', '2019-Q3', '2019-Q4', '2020-Q1', '2020-Q2', '2020-Q3', '2020-Q4', '2021-Q1', '2021-Q2', '2021-Q3', '2021-Q4', '2022-Q1', '2022-Q2', '2022-Q3', '2022-Q4', '2023-Q1']


In [11]:
melted_df = pd.melt(df, id_vars=['GEO'], value_vars=['2010Q1', '2010Q2'], var_name='PERIOD', value_name='ProductionVolume')
melted_df.head(10)

KeyError: "The following 'value_vars' are not present in the DataFrame: ['2010Q1']"